<a href="https://colab.research.google.com/github/JockWang/colab/blob/master/Sea_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install --upgrade tables
!pip install catboost

     |████████████████████████████████| 4.3MB 3.4MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 64.0MB 47kB/s 


In [0]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings
import matplotlib.pyplot as plt
from collections import Counter
import warnings
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_rows', 50)

warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
test = pd.read_hdf('/content/drive/My Drive/Colab Notebooks/Sea/hy_round1_testA_20200102.h5')
train = pd.read_hdf('/content/drive/My Drive/Colab Notebooks/Sea/hy_round1_train_20200102.h5')

In [0]:
def getSecond(x):
  day = str(x).split(' ')
  base = int(day[0])*24*60*60
  hour = int(day[1].split(':')[0])*60*60
  minute = int(day[1].split(':')[1])*60
  second = int(day[1].split(':')[-1])
  return base+hour+minute+second

In [0]:
def base_feature(data,mode='train'):
  train_data = data.sort_values(['ship','time'])
  if mode == 'train':
    type_map = dict(zip(['拖网','围网','刺网'], [0,1,2]))
    train_data['type'] = train_data['type'].map(type_map)
  train_data['second'] = train_data['time'].apply(getSecond)
  train_data = train_data.drop(columns=['time'])
  for col in list(train_data.columns):
    if col not in ['ship','type','time']:
      train_data[col+'_forward'] = train_data.groupby('ship')[col].apply(lambda x: x-x.shift(1))
      if col != 'second':
        train_data[col+'_forward'] = train_data[col+'_forward'].fillna(0)
      else:
        train_data[col+'_forward'] = train_data[col+'_forward'].fillna(600)
  train_data['distance'] = (train_data['v']-train_data['v_forward']/2)*(train_data['second_forward']/3600)
  return train_data
def extract_feature(train_data):
  data = train_data.copy()
  df = data.groupby('ship')['distance','v'].sum()
  df = pd.merge(df, data.groupby('ship')['distance'].count().to_frame(), how='left', on='ship')
  for col in list(data.columns):
    if col not in ['ship','type']:
      temp = data.groupby('ship')[col].agg({col+'_min':'min', col+'_max':'max', col+'_mean':'mean', col+'_std':'std', col+'_skew':'skew'})
      df = pd.merge(df, temp, how='left', on='ship')
    if col == 'type':
      type_df = data.groupby('ship')['type'].mean()
      df = pd.merge(df, type_df, how='left', on='ship')
  # times
  temp = data.groupby('ship')['second'].apply(lambda x: x.max()-x.min()).to_frame()
  df = pd.merge(df, temp, how='left', on='ship')
  df['speed'] = df['distance_x']/df['second']
  df['bias'] = df['v']/df['second']
  df['x_max_x_min'] = df['x_max'] - df['x_min']
  df['y_max_y_min'] = df['y_max'] - df['y_min']
  df['area'] = df['x_max_x_min']*df['y_max_y_min']
  return df.reset_index()

In [0]:
train_data = base_feature(data=train)
test_data = base_feature(data=test,mode='test')

In [0]:
# pd.to_pickle(train_data, '/content/drive/My Drive/Colab Notebooks/Sea/train.pkl')
# pd.to_pickle(test_data, '/content/drive/My Drive/Colab Notebooks/Sea/test.pkl')

In [0]:
train_dataset = extract_feature(train_data=train_data)
test_dataset = extract_feature(train_data=test_data)

In [0]:
train_dataset.head()

,ship,distance_x,v,distance_y,x_min,x_max,x_mean,x_std,x_skew,y_min,y_max,y_mean,y_std,y_skew,v_min,v_max,v_mean,v_std,v_skew,d_min,d_max,d_mean,d_std,d_skew,type,second_min,second_max,second_mean,second_std,second_skew,x_forward_min,x_forward_max,x_forward_mean,x_forward_std,x_forward_skew,y_forward_min,y_forward_max,y_forward_mean,y_forward_std,y_forward_skew,v_forward_min,v_forward_max,v_forward_mean,v_forward_std,v_forward_skew,d_forward_min,d_forward_max,d_forward_mean,d_forward_std,d_forward_skew,second_forward_min,second_forward_max,second_forward_mean,second_forward_std,second_forward_skew,distance_min,distance_max,distance_mean,distance_std,distance_skew,second,speed,bias,x_max_x_min,y_max_y_min,area
0,0,19.471035,110.11,414,6.118352e+06,6.152038e+06,6.119351e+06,5037.320747,5.255558,5.124873e+06,5.130781e+06,5.130494e+06,850.264541,-4.762308,0.0,9.39,0.265966,1.321248,5.520205,0,129,4.613527,21.247770,4.483093,0,95688568,95947099,9.581780e+07,74546.711433,-0.002061,-1.939802,3816.791567,81.368762,420.811680,5.887690,-2820.558404,620.168819,-14.007109,160.276396,-13.645926,-6.80,6.31,0.006256,0.498274,-0.806297,-21.0,113.0,0.246377,6.041196,15.817086,1.0,2343.0,625.920290,168.254710,4.816803,0.0,2.751217,0.047031,0.243856,6.663203,258531,0.000075,0.000426,33686.667453,5907.975523,1.990200e+08
1,1,114.703739,619.05,385,6.049472e+06,6.102450e+06,6.091460e+06,16543.394419,-1.058454,5.042857e+06,5.112874e+06,5.094050e+06,26764.042729,-0.802446,0.0,10.47,1.607922,2.412688,1.590284,0,336,56.153247,91.449382,1.418867,0,95688034,95946021,9.581889e+07,75191.027261,-0.052810,-3521.669135,1645.199578,-68.040756,644.454376,-1.457187,-5621.840326,981.739643,-132.511905,723.286591,-3.455362,-3.19,4.58,0.010364,0.640811,1.466994,-309.0,326.0,0.722078,43.950590,1.072888,50.0,2948.0,671.654545,296.749689,4.362058,0.0,3.037467,0.297932,0.482421,2.247008,257987,0.000445,0.002400,52978.013345,70016.655842,3.709343e+09
2,2,36.114347,138.67,233,6.182482e+06,6.183191e+06,6.183011e+06,207.869601,-2.155218,5.193576e+06,5.193696e+06,5.193682e+06,21.740609,-4.563165,0.0,50.46,0.595150,3.415824,13.631590,0,360,123.356223,123.097127,0.657506,0,96293221,96550918,9.641397e+07,77479.193128,-0.008964,-101.262336,607.572811,2.607609,44.023529,11.249243,-109.260145,109.260145,-0.048519,30.444662,0.004600,-49.97,50.46,0.000901,4.839439,0.140277,-351.0,353.0,-0.699571,161.985432,-0.014550,3.0,3613.0,1108.570815,577.544300,0.470879,0.0,4.562425,0.154997,0.453222,7.172655,257697,0.000140,0.000538,708.835147,120.565000,8.546071e+04
3,3,88.520229,492.90,335,5.228590e+06,5.287805e+06,5.239159e+06,17503.714347,1.608637,4.577467e+06,4.608628e+06,4.601532e+06,11590.605179,-1.194210,0.0,10.09,1.471343,2.528593,2.135446,0,352,121.134328,121.758165,0.469794,0,95688911,95946610,9.582413e+07,72643.328675,-0.165167,-2626.819263,2529.710292,3.758332,752.631229,0.450728,-3988.536391,2813.795154,5.329341,573.369647,-0.094017,-8.58,9.39,-0.004030,1.613107,0.214012,-328.0,335.0,-0.629851,117.016557,0.038799,1.0,3025.0,771.041791,469.420663,1.288274,0.0,1.666940,0.264239,0.408792,1.904609,257699,0.000344,0.001913,59214.738740,31160.661097,1.845170e+09
4,4,104.498388,566.30,401,7.049394e+06,7.070797e+06,7.062005e+06,5979.578887,-0.596732,6.094996e+06,6.136033e+06,6.116389e+06,12055.148984,-0.331618,0.0,10.09,1.412219,2.496836,1.910336,0,359,139.067332,121.130025,0.372601,1,96292812,96551844,9.642151e+07,75855.456648,0.010843,-2603.619533,3556.700658,-9.382649,585.334897,-0.224270,-2354.712763,4910.804362,20.965048,673.238864,2.093895,-7.40,9.39,0.000549,2.028397,0.356495,-351.0,356.0,-0.019950,151.983156,0.044284,537.0,2453.0,647.461347,186.231305,4.732627,0.0,3.107556,0.260594,0.442751,2.319658,259032,0.000403,0.002186,21402.484584,41036.883038,8.782913e+08


In [0]:
features = [col for col in list(train_dataset.columns) if col not in ['type']]
x_middle, y_middle = train_dataset[features], train_dataset['type']
x_final, y_final = SMOTE(random_state=2020).fit_sample(x_middle, y_middle)
print(x_final.shape,y_final.shape)
test_final = test_dataset[features]

(13083, 65) (13083,)


In [0]:
# x_final.head()

In [0]:
def selectMost(preds):
  res = []
  for i in range(preds.shape[1]):
    res.append(Counter(preds[:,i]).most_common(1)[0][0])
  return np.array(res)
def finalRes(res):
  result = []
  for r in res:
    result.append(yu_type[r])
  return result

In [0]:
params = {
  'objective': 'multiclass',
  'num_class': 3,
  'learning_rate': 0.01,
  'num_leaves': 31,
  'bagging_fraction': .8,
  'feature_fraction': .8,
  'lambda_l1': 0,
  'lambda_l2': 0.5,
  'early_stopping_rounds': 100,
}

In [0]:
def LGBKfoldResult(train=x_final,test=test_final,label=y_final,params=params,K=20):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      # train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      train_x,valid_x,train_y,valid_y = train[train_index],train[valid_index],label[train_index],label[valid_index]
      lgb_train = lgb.Dataset(train_x,train_y)
      lgb_valid = lgb.Dataset(valid_x,valid_y)
      model = lgb.train(params,train_set=lgb_train,valid_sets=lgb_valid,num_boost_round=5000,early_stopping_rounds=50, verbose_eval=50)
      models.append(model)
      val_pred = model.predict(valid_x,num_iteration=model.best_iteration).argmax(axis=1)
      print(metrics.f1_score(valid_y, val_pred, average='macro'))
      preds.append(model.predict(test,num_iteration=model.best_iteration).argmax(axis=1))
  preds = np.array(preds)
  return models,preds.astype(int)

In [0]:
models, lgb_preds = LGBKfoldResult()

[4400]	valid_0's multi_logloss: 0.131264
[4450]	valid_0's multi_logloss: 0.131038
[4500]	valid_0's multi_logloss: 0.130755
[4550]	valid_0's multi_logloss: 0.130615
[4600]	valid_0's multi_logloss: 0.130419
[4650]	valid_0's multi_logloss: 0.130138
[4700]	valid_0's multi_logloss: 0.129951
[4750]	valid_0's multi_logloss: 0.129672
[4800]	valid_0's multi_logloss: 0.129466
[4850]	valid_0's multi_logloss: 0.129292
[4900]	valid_0's multi_logloss: 0.129341
[4950]	valid_0's multi_logloss: 0.129335
Early stopping, best iteration is:
[4854]	valid_0's multi_logloss: 0.129266
0.9539956623289957
第 13 个Fold
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_logloss: 0.800993
[100]	valid_0's multi_logloss: 0.639309
[150]	valid_0's multi_logloss: 0.541364
[200]	valid_0's multi_logloss: 0.476898
[250]	valid_0's multi_logloss: 0.430513
[300]	valid_0's multi_logloss: 0.396905
[350]	valid_0's multi_logloss: 0.370666
[400]	valid_0's multi_logloss: 0.348003
[450]	valid_0's mult

In [0]:
res = finalRes(selectMost(lgb_preds))
sub = test_dataset[['ship']]
sub['pred'] = res
sub.to_csv('/content/drive/My Drive/result_lgb.csv',index=False,header=False,encoding='utf_8_sig')

In [0]:
xgb_params = {
    'learning_rate': 0.01,
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'num_class': 3,
    'max_depth': 7,
    'subsample': .7,
    'colsample_bytree': .7,
    'colsample_bylevel': .7,
    'lambda': 0.5,
}

In [0]:
def XGBKfoldResult(train=x_final,test=test_final,label=y_final,params=xgb_params,K=20):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  # test = xgb.DMatrix(test.copy())
  test = xgb.DMatrix(test.values.copy())
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      # train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      train_x,valid_x,train_y,valid_y = train[train_index],train[valid_index],label[train_index],label[valid_index]
      xgb_train = xgb.DMatrix(train_x,train_y)
      xgb_valid = xgb.DMatrix(valid_x,valid_y)
      evallist = [(xgb_valid,'eval')]
      model = xgb.train(params,dtrain=xgb_train,num_boost_round=5000,early_stopping_rounds=100,evals=evallist,verbose_eval=50)
      models.append(model)
      val_pred = model.predict(xgb_valid).reshape((-1))
      print(metrics.f1_score(valid_y, val_pred, average='macro'))
      preds.append(model.predict(test).reshape((-1)))
  preds = np.array(preds)
  return models,preds.astype(int)

In [0]:
xgb_models, xgb_preds = XGBKfoldResult()

第 0 个Fold
[0]	eval-mlogloss:1.08966
Will train until eval-mlogloss hasn't improved in 100 rounds.
[50]	eval-mlogloss:0.768116
[100]	eval-mlogloss:0.591639
[150]	eval-mlogloss:0.483887
[200]	eval-mlogloss:0.413661
[250]	eval-mlogloss:0.365531
[300]	eval-mlogloss:0.330821
[350]	eval-mlogloss:0.305635
[400]	eval-mlogloss:0.286502
[450]	eval-mlogloss:0.271124
[500]	eval-mlogloss:0.259352
[550]	eval-mlogloss:0.249044
[600]	eval-mlogloss:0.240794
[650]	eval-mlogloss:0.233022
[700]	eval-mlogloss:0.22598
[750]	eval-mlogloss:0.219988
[800]	eval-mlogloss:0.214407
[850]	eval-mlogloss:0.209387
[900]	eval-mlogloss:0.204976
[950]	eval-mlogloss:0.200666
[1000]	eval-mlogloss:0.197225
[1050]	eval-mlogloss:0.193756
[1100]	eval-mlogloss:0.190708
[1150]	eval-mlogloss:0.187228
[1200]	eval-mlogloss:0.184359
[1250]	eval-mlogloss:0.181814
[1300]	eval-mlogloss:0.179336
[1350]	eval-mlogloss:0.176936
[1400]	eval-mlogloss:0.174646
[1450]	eval-mlogloss:0.17242
[1500]	eval-mlogloss:0.170376
[1550]	eval-mlogloss:0.1

In [0]:
yu_type = {0:'拖网',1:'围网',2:'刺网'}
res = finalRes(selectMost(np.vstack((xgb_preds,lgb_preds))))
sub = test_dataset[['ship']]
sub['pred'] = res
sub.to_csv('/content/drive/My Drive/result_xgb_lgb_2.csv',index=False,header=False,encoding='utf_8_sig')

In [0]:
print('拖网:', 1249/sub.shape[0])
print('围网:', 481/sub.shape[0])
print('刺网:', 270/sub.shape[0])
Counter(sub['pred'])

拖网: 0.6245
围网: 0.2405
刺网: 0.135


Counter({'刺网': 270, '围网': 481, '拖网': 1249})

In [0]:
cat_features = []
def CatKfoldResult(train=x_final,test=test_final,label=y_final,cat_features=cat_features,K=5):
  fold = StratifiedKFold(n_splits=K,shuffle=True)
  preds = []
  models = []
  i = 0
  for train_index, valid_index in fold.split(train,label):
      print('第',str(i),'个Fold')
      i += 1
      # train_x,valid_x,train_y,valid_y = train.iloc[train_index],train.iloc[valid_index],label[train_index],label[valid_index]
      train_x,valid_x,train_y,valid_y = train[train_index],train[valid_index],label[train_index],label[valid_index]
      eval_set = cat.Pool(data=valid_x,label=valid_y,cat_features=cat_features)
      model = cat.CatBoostClassifier(iterations=5000, learning_rate=0.1, depth=7, loss_function='MultiClass')
      model.fit(train_x,train_y,use_best_model=True,eval_set=eval_set,cat_features=cat_features,early_stopping_rounds=100,verbose_eval=50)
      models.append(model)
      val_pred = model.predict(valid_x).reshape((-1))
      print(metrics.f1_score(valid_y, val_pred, average='macro'))
      preds.append(model.predict(test).reshape((-1)))
  
  preds = np.array(preds)
  return models,preds.astype(int)

In [0]:
cat_model, cat_preds = CatKfoldResult()

第 0 个Fold
0:	learn: 1.0128294	test: 1.0147437	best: 1.0147437 (0)	total: 115ms	remaining: 9m 37s
50:	learn: 0.4426961	test: 0.4812284	best: 0.4812284 (50)	total: 4.2s	remaining: 6m 47s
100:	learn: 0.3487435	test: 0.4063151	best: 0.4063151 (100)	total: 8.37s	remaining: 6m 46s
150:	learn: 0.2881353	test: 0.3610745	best: 0.3610745 (150)	total: 12.4s	remaining: 6m 37s
200:	learn: 0.2469707	test: 0.3343860	best: 0.3343860 (200)	total: 16.4s	remaining: 6m 30s
250:	learn: 0.2119149	test: 0.3103364	best: 0.3103364 (250)	total: 20.3s	remaining: 6m 24s
300:	learn: 0.1871538	test: 0.2946324	best: 0.2946324 (300)	total: 24.4s	remaining: 6m 20s
350:	learn: 0.1659371	test: 0.2801876	best: 0.2801876 (350)	total: 28.4s	remaining: 6m 16s
400:	learn: 0.1486965	test: 0.2694206	best: 0.2694206 (400)	total: 32.6s	remaining: 6m 13s
450:	learn: 0.1344110	test: 0.2610661	best: 0.2610661 (450)	total: 36.6s	remaining: 6m 8s
500:	learn: 0.1228347	test: 0.2532594	best: 0.2532594 (500)	total: 40.6s	remaining: 6m 4

In [0]:
yu_type = {0:'拖网',1:'围网',2:'刺网'}
res = finalRes(selectMost(cat_preds))
sub = test_dataset[['ship']]
sub['pred'] = res
sub.to_csv('/content/drive/My Drive/result_cat.csv',index=False,header=False,encoding='utf_8_sig')